### SQLite Database Initialization

This section initializes the SQLite database by creating necessary tables such as Items, Auctions, and ActionEvents. It ensures the existence of the required tables and sets up the database schema.

### File and Configuration Setup

Additionally, it creates other essential files and configurations needed for subsequent operations.

In [3]:
import sqlite3

db_path = 'C:/sqlite3/auction.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Items (
        item_id INT PRIMARY KEY,
        item_name TEXT,
        quality TEXT,
        item_level INT,
        required_level INT,
        item_class TEXT,
        item_subclass TEXT,
        purchase_price_gold INT,
        purchase_price_silver INT,
        sell_price_gold INT,
        sell_price_silver INT,
        max_count INT,
        is_stackable INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Auctions (
        auction_id INT PRIMARY KEY,
        bid INT,
        buyout INT,
        quantity INT,
        time_left TEXT,
        item_id INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS ActionEvents (
        auction_id INT,
        record DATETIME,
        PRIMARY KEY (auction_id, record),
        FOREIGN KEY (auction_id) REFERENCES Auctions(auction_id)
    )
''')

conn.commit()
conn.close()


### MySQL Items Data Retrieval

In this part, the script retrieves data from a MySQL database. It reads the MySQL database configuration from a JSON file, establishes a connection, and fetches data from the Items table. The retrieved data is then loaded into a Pandas DataFrame.

### SQLite Database Update

After retrieving the data, the script connects to the SQLite database, deletes all existing records from the Items table, and appends the newly fetched data.


In [4]:

import json
import os
from datetime import datetime
from tqdm import tqdm
import sqlite3

data_dir = 'C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator'
db = sqlite3.connect(db_path)
cursor = db.cursor()

for root, dirs, files in os.walk(data_dir):
    for filename in tqdm(files):
        if filename.endswith(".json") and filename != 'config.json':
            filepath = os.path.join(root, filename)
            try:
                data = json.load(open(filepath, "r"))
            except (FileNotFoundError, json.JSONDecodeError) as e:
                print(f"Error reading file {filepath}: {e}")
                continue

            print(f"Processing file: {filepath}")

            auction_record = datetime.strptime(filename[:-5], "%Y%m%dT%H")

            auctions_data = [
                (auction["id"], auction["bid"], auction["buyout"], auction["quantity"], auction["time_left"], auction["item"]["id"])
                for auction in data["auctions"]
            ]
            action_events_data = [(auction["id"], auction_record.strftime('%Y-%m-%d %H:%M:%S')) for auction in data["auctions"]]

            try:
                cursor.executemany("""
                    INSERT OR REPLACE INTO Auctions (auction_id, bid, buyout, quantity, time_left, item_id)
                    VALUES (?, ?, ?, ?, ?, ?)
                """, auctions_data)
                db.commit()
                print(f"Auction data from file {filepath} successfully inserted into Auctions.")
            except sqlite3.Error as err:
                db.rollback()
                print(f"Error inserting auction data for file {filepath} in Auctions: {err}")

            try:
                cursor.executemany("""
                    INSERT OR REPLACE INTO ActionEvents (auction_id, record)
                    VALUES (?, ?)
                """, action_events_data)
                db.commit()
                print(f"Auction events for file {filepath} successfully inserted in ActionEvents.")
            except sqlite3.Error as err:
                db.rollback()
                print(f"Error inserting auction events for file {filepath} in ActionEvents: {err}")

cursor.close()
db.close()


100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

Processing file: C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\09-01-2024\20240109T00.json
Auction data from file C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\09-01-2024\20240109T00.json successfully inserted into Auctions.


  4%|███▍                                                                               | 1/24 [00:20<07:47, 20.32s/it]

Auction events for file C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\09-01-2024\20240109T00.json successfully inserted in ActionEvents.
Processing file: C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\09-01-2024\20240109T01.json
Auction data from file C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\09-01-2024\20240109T01.json successfully inserted into Auctions.


  8%|██████▉                                                                            | 2/24 [00:30<05:20, 14.57s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 23/24 [04:36<00:11, 11.04s/it]

Auction events for file C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\10-01-2024\20240110T22.json successfully inserted in ActionEvents.
Processing file: C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\10-01-2024\20240110T23.json
Auction data from file C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\10-01-2024\20240110T23.json successfully inserted into Auctions.


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [04:47<00:00, 11.98s/it]


Auction events for file C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\10-01-2024\20240110T23.json successfully inserted in ActionEvents.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


### Auction Data and Items Storage

This section processes JSON files containing auction data. It iterates through the files, extracts relevant information, and inserts it into the Auctions and ActionEvents tables of the SQLite database.

### Data Import from Pandas DataFrame

Moreover, it initializes a connection to the SQLite database and imports data from a Pandas DataFrame into the Items table. This DataFrame is generated from MySQL database retrieval, ensuring the SQLite Items table is up-to-date.

In [18]:
import pandas as pd
import mysql.connector
import json
import sqlite3

with open('C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator/config.json') as f:
    config = json.load(f)
    print(config)

db_path = 'C:/sqlite3/auction.db'
query = "SELECT * FROM Items"

def import_data():
    try:
        mysql_db = mysql.connector.connect(**config['database'])
        print("Conexión exitosa a la base de datos MySQL.")
    except mysql.connector.Error as err:
        print(f"Error al conectarse a la base de datos MySQL: {err}")
        return

    df = pd.read_sql_query(query, mysql_db)
    mysql_db.close()

    try:
        conn = sqlite3.connect(db_path)
        print("Conexión exitosa a la base de datos SQLite.")
    except sqlite3.Error as err:
        print(f"Error al conectarse a la base de datos SQLite: {err}")
        return

    with conn.cursor() as cursor:
        cursor.execute("DELETE FROM Items")
        conn.commit()
        print("Datos borrados de la tabla Items.")

        df.to_sql('Items', conn, if_exists='append', index=False)
        print("Datos insertados en la tabla Items.")

    conn.close()

if __name__ == '__main__':
    import_data()



JSONDecodeError: Expecting property name enclosed in double quotes: line 10 column 1 (char 150)

In [2]:
import sqlite3

# Conectarse a la base de datos SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para obtener datos (ejemplo limitado a 100)
query_2 = """
    SELECT
        a.auction_id,
        a.bid / 10000 AS bid_in_gold,
        a.buyout / 10000 AS buyout_in_gold,
        (a.buyout / 10000) / a.quantity AS unit_price,
        a.quantity,
        a.time_left,
        a.item_id,
        i.item_name,
        i.quality,
        i.item_class,
        i.item_subclass,
        i.is_stackable,
        i.purchase_price_gold,
        i.required_level,
        i.item_level,
        i.sell_price_gold,
        MIN(ae.record) AS first_appearance_timestamp,
        strftime('%Y', MIN(ae.record)) AS first_appearance_year,
        strftime('%m', MIN(ae.record)) AS first_appearance_month,
        strftime('%d', MIN(ae.record)) AS first_appearance_day,
        strftime('%H', MIN(ae.record)) AS first_appearance_hour,
        COUNT(*) AS hours_on_sale
    FROM Auctions a
    JOIN ActionEvents ae ON a.auction_id = ae.auction_id
    JOIN Items i ON i.item_id = a.item_id
    GROUP BY a.auction_id
"""

# Ejecutar la consulta y obtener los resultados
cursor.execute(query_2)
results_2 = cursor.fetchall()

# Cerrar la conexión a la base de datos
conn.close()


NameError: name 'db_path' is not defined

In [13]:
import pandas as pd

# Crear un DataFrame con los resultados
df = pd.DataFrame(results_2, columns=[
    "auction_id", "bid_in_gold", "buyout_in_gold", "unit_price", "quantity", "time_left",
    "item_id", "item_name", "quality", "item_class", "item_subclass", "is_stackable",
    "purchase_price_gold", "required_level", "item_level", "sell_price_gold",
    "first_appearance_timestamp", "first_appearance_year", "first_appearance_month",
    "first_appearance_day", "first_appearance_hour", "hours_on_sale"
])

# Importar funciones de preprocesamiento
from preprocess_data import preprocess_data

# Aplicar preprocesamiento a los datos
df_processed = preprocess_data(df)


NameError: name 'results_2' is not defined

In [14]:
import pickle
from sklearn.metrics import mean_squared_error

# Cargar el modelo desde el archivo .pkl
with open('linear_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Hacer predicciones en los datos preprocesados
predictions = model.predict(df_processed)

# Calcular RMSE
rmse = mean_squared_error(df_processed['target_column'], predictions, squared=False)

# Imprimir el resultado
print("RMSE:", rmse)


FileNotFoundError: [Errno 2] No such file or directory: 'linear_model.pkl'